# Hyper parameters optimization #

## Parameters cell ##

Parameters are overiddent by papermill when run inside DVC stages



In [1]:
n_folds = 5 # this sets global setting of which how many bootstraps to use
n_hold_out = 1
repeats = 10
n_trials = 100
importance_type = "split"
trait = "lifespan"#"lifespan"

debug_local = True #to use local version

In [2]:
from pathlib import Path
import sys
import inspect

local = (Path("..") / "yspecies").resolve()
if debug_local and local.exists():
  sys.path.insert(0, Path("..").as_posix())
  #sys.path.insert(0, local.as_posix())
  print("extending pathes with local yspecies")
  print(sys.path)
  %load_ext autoreload
  %autoreload 2

extending pathes with local yspecies
['..', '/data/sources/yspecies/notebooks', '/data/miniconda3/envs/yspecies/lib/python38.zip', '/data/miniconda3/envs/yspecies/lib/python3.8', '/data/miniconda3/envs/yspecies/lib/python3.8/lib-dynload', '', '/data/miniconda3/envs/yspecies/lib/python3.8/site-packages', '/data/miniconda3/envs/yspecies/lib/python3.8/site-packages/IPython/extensions', '/home/anton/.ipython']


In [3]:
from dataclasses import dataclass, replace
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
from typing import *
from yspecies.dataset import *
from yspecies.utils import *
from yspecies.workflow import TupleWith, Repeat, Collect
from yspecies.config import *
from yspecies.preprocess import FeatureSelection, DataExtractor
from yspecies.partition import DataPartitioner, PartitionParameters
from yspecies.selection import ShapSelector
from yspecies.tuning import Tune, MultiObjectiveResults
from yspecies.models import ResultsCV, CrossValidator
from yspecies.results import FeatureSummary
import optuna
from optuna import Study, Trial

In [5]:
#settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import pprint
pp = pprint.PrettyPrinter(indent=4)

#charts settings
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
%matplotlib auto
plt.ioff()
set_matplotlib_formats('svg')

Using matplotlib backend: agg


### Loading data ###
Let's load data from species/genes/expressions selected by select_samples.py notebook

In [6]:
from pathlib import Path
locations: Locations = Locations("./") if Path("./data").exists() else Locations("../")

In [7]:
from loguru import logger

logger.add(locations.logs / "optimization_stage_2_errors.log", backtrace=True, diagnose=True)
logger.add(locations.logs / "optimization_stage_2.log", rotation="12:00")     # New file is created each day at noon

2

In [8]:
data = ExpressionDataset.from_folder(locations.interim.dir / "stage_2_input")
data

expressions,genes,species,samples,Genes Metadata,Species Metadata
"(408, 142)","(142, 37)",38,408,"(142, 2)","(38, 18)"


In [9]:
not_validated_species = data.min_max_trait(trait)
not_validated_species

['Mus_caroli', 'Homo_sapiens']

In [10]:
selection = FeatureSelection(
    samples = ["tissue","species"], #samples metadata to include
    species =  [], #species metadata other then Y label to include
    exclude_from_training = ["species"],  #exclude some fields from LightGBM training
    to_predict = trait, #column to predict
    categorical = ["tissue"],
    select_by = "shap",
    importance_type =  importance_type,
    feature_perturbation = "tree_path_dependent",
    not_validated_species = not_validated_species
)
selection

Samples metadata,Species metadata,Genes,Predict label,not_validated species
"['tissue', 'species']",[],all,lifespan,"['Mus_caroli', 'Homo_sapiens']"


## Setting up ShapSelector ##

Deciding on selection parameters (which fields to include, exclude, predict)

In [11]:
partition_params = PartitionParameters(n_folds, n_hold_out, 2,   42)

In [12]:
def load_study(trait: str):
    url = f'sqlite:///' +str((locations.interim.optimization / (trait+"_2.sqlite")).absolute())
    print('loading (if exists) study from '+url)
    storage = optuna.storages.RDBStorage(
        url=url
        #engine_kwargs={'check_same_thread': False}
    )
    return optuna.multi_objective.study.create_study(directions=['maximize','minimize','maximize'], storage = storage, study_name = f"{trait}_r2_huber_kendall", load_if_exists = True)

study = load_study(trait)
study
#metrics, params = MultiObjectiveResults.from_study(study).best_metrics_params_r2()
#metrics, params

loading (if exists) study from sqlite:////data/sources/yspecies/notebooks/../data/interim/optimization/lifespan_2.sqlite


create_study is experimental (supported from v1.4.0). The interface can change in the future.
NSGAIIMultiObjectiveSampler is experimental (supported from v1.5.0). The interface can change in the future.
RandomMultiObjectiveSampler is experimental (supported from v1.4.0). The interface can change in the future.
[I 2020-09-11 17:07:44,096] Using an existing study with name 'lifespan_r2_huber_kendall' instead of creating a new one.
MultiObjectiveStudy is experimental (supported from v1.4.0). The interface can change in the future.


In [13]:
def objective_parameters(trial: Trial) -> dict:
    return {
        'objective': 'regression',
        'metric': {'mae', 'mse', 'huber'},
        'verbosity': -1,
        'boosting_type': trial.suggest_categorical('boosting_type', ['dart', 'gbdt']),
        'lambda_l1': trial.suggest_uniform('lambda_l1', 0.01, 3.0),
        'lambda_l2': trial.suggest_uniform('lambda_l2', 0.01, 3.0),
        'max_leaves': trial.suggest_int("max_leaves", 15, 25),
        'max_depth': trial.suggest_int('max_depth', 3, 8),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.3, 1.0),
        'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.1),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 3, 8),
        'drop_rate': trial.suggest_uniform('drop_rate', 0.1, 0.3),
        "verbose": -1
    }
optimization_parameters = objective_parameters

In [14]:
from yspecies.workflow import SplitReduce

def side(i: int):
    print(i)
    return i

prepare_partition = SplitReduce(
    outputs = DataPartitioner(), 
    split = lambda x: [(x[0], replace(partition_params, seed=side(x[2])))], 
    reduce = lambda x, output: (output[0], x[1]) 
)                               
partition_and_cv = Pipeline(
    [
        ("prepare partition", prepare_partition),
        ("shap_computation", ShapSelector()) #('crossvalidator', CrossValidator())        
    ]
)

partition_and_cv_repeat =  Pipeline([
    ("repeat_cv_pipe", Repeat(partition_and_cv, repeats, lambda x, i: [x[0], x[1], i] )),
    #("collect_mean", Collect(fold=lambda results: np.array([r.last("huber") for r in results]).mean()))
    ("collect_mean", Collect(fold=lambda results: (FeatureSummary(results).metrics_average.R2, FeatureSummary(results).metrics_average.huber, FeatureSummary(results).kendall_tau_abs_mean)))    
    ]
    )

p = Pipeline([
     ('extractor', DataExtractor()),
     ('tune', Tune(partition_and_cv_repeat, study = study, n_trials = n_trials, parameters_space = optimization_parameters))    
])

In [ ]:
results = p.fit_transform((data, selection))
results

MultiObjectiveStudy is experimental (supported from v1.4.0). The interface can change in the future.
FrozenMultiObjectiveTrial is experimental (supported from v1.4.0). The interface can change in the future.
MultiObjectiveStudy is experimental (supported from v1.4.0). The interface can change in the future.
FrozenMultiObjectiveTrial is experimental (supported from v1.4.0). The interface can change in the future.
FrozenMultiObjectiveTrial is experimental (supported from v1.4.0). The interface can change in the future.
MultiObjectiveTrial is experimental (supported from v1.4.0). The interface can change in the future.
MultiObjectiveStudy is experimental (supported from v1.4.0). The interface can change in the future.
MultiObjectiveStudy is experimental (supported from v1.4.0). The interface can change in the future.
FrozenMultiObjectiveTrial is experimental (supported from v1.4.0). The interface can change in the future.


0


2020-09-11 17:07:45.916 | INFO     | yspecies.selection:fit:78 - ===== fitting models with seed 0 =====
2020-09-11 17:07:45.918 | INFO     | yspecies.selection:fit:79 - PARAMETERS:
{'objective': 'regression', 'metric': {'mse', 'huber', 'mae'}, 'verbosity': -1, 'boosting_type': 'gbdt', 'lambda_l1': 0.806868772761894, 'lambda_l2': 1.361676460911139, 'max_leaves': 18, 'max_depth': 3, 'feature_fraction': 0.5479409728228775, 'bagging_fraction': 0.6212877919288045, 'learning_rate': 0.022559688029803765, 'min_data_in_leaf': 8, 'drop_rate': 0.27117231134759434, 'verbose': -1}
2020-09-11 17:07:45.922 | INFO     | yspecies.selection:fit:82 - SEED: 0 | FOLD: 0 | VALIDATION_SPECIES: ['Gorilla_gorilla', 'Pan_paniscus']


[LightGBM] [Warning] verbosity is set=-1, verbose=-1 will be ignored. Current value: verbosity=-1
Training until validation scores don't improve for 10 rounds
[250]	valid_0's l2: 200.78	valid_0's huber: 7.13187	valid_0's l1: 8.33072
Did not meet early stopping. Best iteration is:
[250]	valid_0's l2: 200.78	valid_0's huber: 7.13187	valid_0's l1: 8.33072


2020-09-11 17:07:46.173 | INFO     | yspecies.selection:fit:82 - SEED: 0 | FOLD: 1 | VALIDATION_SPECIES: ['Heterocephalus_glaber', 'Capra_hircus']


...

Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[183]	valid_0's l2: 36.5499	valid_0's huber: 2.70962	valid_0's l1: 3.36722


2020-09-11 19:46:02.787 | INFO     | yspecies.selection:fit:82 - SEED: 8 | FOLD: 2 | VALIDATION_SPECIES: ['Ursus_americanus', 'Capra_hircus']


Training until validation scores don't improve for 10 rounds
[250]	valid_0's l2: 32.4381	valid_0's huber: 2.40957	valid_0's l1: 3.04373
Did not meet early stopping. Best iteration is:
[250]	valid_0's l2: 32.4381	valid_0's huber: 2.40957	valid_0's l1: 3.04373


2020-09-11 19:46:02.977 | INFO     | yspecies.selection:fit:82 - SEED: 8 | FOLD: 3 | VALIDATION_SPECIES: ['Bos_taurus', 'Rhinolophus_ferrumequinum']


Training until validation scores don't improve for 10 rounds
[250]	valid_0's l2: 38.7399	valid_0's huber: 2.34964	valid_0's l1: 2.9807
Did not meet early stopping. Best iteration is:
[246]	valid_0's l2: 38.7297	valid_0's huber: 2.35058	valid_0's l1: 2.98173


In [ ]:
results.best_trial_r2().values #before: 0.92651

In [64]:
results.best_trial_r2().values #before: 0.92

(0.9265172335576978, 3.108940798535765, 0.5238496205639561)

In [66]:
results.best_trial_r2().params

{'boosting_type': 'gbdt',
 'lambda_l1': 1.0074040471792671,
 'lambda_l2': 2.3500616173039672,
 'max_leaves': 19,
 'max_depth': 3,
 'feature_fraction': 0.5168464794482679,
 'bagging_fraction': 0.42259626867860933,
 'learning_rate': 0.04901611652323495,
 'min_data_in_leaf': 8,
 'drop_rate': 0.16401872685861194}